### Eventloop

Ein Eventloop is eine Schleife die z.B all 100 Millisekunden prüft
Um (z.B. Eingaben während eines Spiels) und bearbeitet es (hier schieben wir die Elemente der 

***
**threading.Event**:  
Eine Klasse mit  Methoden

- `is_set()` : liefert `True` oder `False` (Default).
- `set()`    : `is_set()` liefert nun `True`
- `clear()`  : `is_set()` liefert nun `False`
***

In [6]:
import threading


stop_event = threading.Event()
stop_event.is_set()

False

In [7]:
stop_event.set()
stop_event.is_set()

True

In [8]:
stop_event.clear()
stop_event.is_set()

False

In [42]:
def f():
    print('Sorry fuer die Verspaetung')


thread = threading.Timer(1, f)
thread.start()
print('Wait ...')

Wait ...
Sorry fuer die Verspaetung


***
Mit  `threading.Timer` kann ein Event-Loop implementiert werden:
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.



Der Thread entfernt das letzte Element der Liste `event_queue`.
Ist der `event_queue` leer, wird das letzte Event verwendet.
***

In [40]:
import threading
from ipywidgets import Output, Button
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)
button_stop = Button(description='Stop')

state = {'pos': (50, 50), 'count': 0}
directions = {'Up': (0, -5), 'Down': (0, 5), 'Left': (-5, 0), 'Right': (5, 0)}

stop_event = threading.Event()
event_queue = []


@out.capture()
def on_key_down(key, *flags):
    if key.startswith('Arrow'):
        event_queue.insert(0, key[5:])
    print(key)


def event_loop(last_event=None):
    if event_queue:
        event = event_queue.pop()
    else:
        event = last_event

    dx, dy = directions.get(event, (0, 0))
    x0, y0 = state['pos']
    x, y = (x0+dx, y0+dy)

    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)
    state['count'] += 1

    if x < 0 or y < 0 or x > 100 or y > 100:
        canvas.fill_text('Game over!', 10, 50)
        stop_event.set()

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(event,))
        thread.name = f'Eventloop-{state['count']}'
        thread.start()


canvas.on_key_down(on_key_down)
button_stop.on_click(lambda bt: stop_event.set())
display(canvas, button_stop, out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Button(description='Stop', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [41]:
event_queue.insert(0, 'Right')
event_loop()
canvas.focus()  # Canvas Widget hoert auf Events, ohne das se zuerst angeklickt werden muss

In [32]:
# lauft der Eventloop Thred noch?
[thread.name for thread in threading.enumerate()[8:]]

['Eventloop-6']

### Aufgabe
Erstellen einen weitere Button `New Game`. Beim Klicken auf diesen Button soll eine 
Funktion `new_game()` aufgerufen werden, welche
- die Leinwand löscht,
- `stop_event.clear()` aufruft,
- den Event-Queue gelöscht,
- `'Right'` in Event-Queue einfügt,
- den Event-Loop startet und der Leinwand Fokus gibt.  

Leite Fehlermeldungen der Funktion `new_game()` ins Output-Widget `out` um.